**CII4Q3 Visi Komputer**

---



#Exercise 5a: Klasifikasi Fruits

---

Ichwan Rizky Wahyudin(1301213434)\
Abdul Wasiul Khair(1301213278)

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR, class_names):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=45,
        width_shift_range=.25,
        height_shift_range=.25,
        shear_range=.25,
        zoom_range=.25,
        fill_mode='nearest',
        horizontal_flip=True
    )

    train_generator = train_datagen.flow_from_directory(
        directory=TRAINING_DIR,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        classes=class_names
    )

    validation_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 40, 
        width_shift_range = .25,
        height_shift_range = .25,
        shear_range = .25, 
        zoom_range = .25,
        horizontal_flip = True, 
        fill_mode = 'nearest')

    validation_generator = validation_datagen.flow_from_directory(
        directory=VALIDATION_DIR,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        classes=class_names
    )
    return train_generator, validation_generator

class_names = os.listdir('dataset/train')
TRAINING_DIR = './dataset/train'
VALIDATION_DIR = './dataset/test'
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR, class_names)


: 

In [ ]:
import matplotlib.pyplot as plt
import random

def plot_images(images, labels, class_names):
    plt.figure(figsize=(10, 10))
    num_images = min(len(images), len(labels))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        idx = random.randint(0, num_images - 1)
        plt.imshow(images[idx])
        plt.title(class_names[np.argmax(labels[idx])])
        plt.axis("off")
    plt.show()

images, labels = next(train_generator)
plot_images(images, labels, class_names)


: 

In [ ]:
def count_data(directory):
    count = {}
    for class_name in class_names:
        count[class_name] = len(os.listdir(os.path.join(directory, class_name)))
    return count

train_count = count_data(TRAINING_DIR)
validation_count = count_data(VALIDATION_DIR)
print(train_count)
print(validation_count)

: 

In [ ]:
def plot_data_distribution(train_count, validation_count):
    fig, ax = plt.subplots()
    bar_width = 0.3
    opacity = 0.8
    index = np.arange(len(class_names))
    train_counts = [train_count[class_name] for class_name in class_names]
    validation_counts = [validation_count[class_name] for class_name in class_names]
    train_bar = plt.bar(index, train_counts, bar_width, alpha=opacity, color='b', label='Train')
    validation_bar = plt.bar(index + bar_width, validation_counts, bar_width, alpha=opacity, color='r', label='Validation')
    fig.set_size_inches(10, 6)
    plt.xlabel('Class')
    plt.ylabel('Count')
    plt.title('Cek keseimbangan data setiap kelas')
    plt.xticks(index + bar_width, class_names)
    plt.legend()
    plt.tight_layout()
    
plot_data_distribution(train_count, validation_count)

: 

In [ ]:
pretrained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3),
                                                include_top=False,
                                                weights='imagenet')

for layer in pretrained_model.layers:
        layer.trainable=False
    
model = tf.keras.models.Sequential([
    pretrained_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

: 

In [ ]:
pretrained_model.summary()

: 

In [ ]:
history = model.fit(train_generator,
                              epochs=15,
                              validation_data=validation_generator,
                              validation_steps=1,
                              verbose=2)

: 

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

: 

In [ ]:
import os
import numpy as np
from keras.preprocessing import image

path = './dataset/predict'
img = 'f2.jpeg'
img_path = os.path.join(path, img)
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)

plt.imshow(img)
plt.title(class_names[np.argmax(classes[0])])
plt.show()
print(classes)
    


: 